In [37]:
%matplotlib inline

import pyfits
import numpy as np
import matplotlib
matplotlib.rcParams['font.size'] = 15
from matplotlib import pyplot as plt
import sys
sys.path.append('../')
import photPack2
from astropy.time import Time
import emcee
import matplotlib.cm as cm
dophotometry = True
dofitting = False


if dophotometry:
    print 'Calculating photometry'

    wasp6paths_nodsub_odd = ["%s%04d%s" % ('/local/tmp/mosfire/2014sep18_analysis/m140918_', \
                             i,'n_nobadpxl.fits') for i in range(365,629,1)]
    wasp6paths_sum_odd = ["%s%04d%s" % ('/local/tmp/mosfire/2014sep18_analysis/m140918_',\
                          i,'sum_nobadpxl.fits') for i in range(365,629,1)]
    testimgpath = wasp6paths_nodsub_odd[8]
    testimg = pyfits.getdata(testimgpath)
    arcpath = '/local/tmp/mosfire/2014sep18_analysis/m140918_0005shifted.fits'
    arcimage = pyfits.getdata(arcpath)
    wavelengthsoln = np.load('wavelengthsoln.npy')

    times = np.zeros(len(wasp6paths_nodsub_odd))
    fluxes = np.zeros((len(wasp6paths_nodsub_odd), 2))
    centroids = np.zeros((len(wasp6paths_nodsub_odd), 2))
    airmass = np.zeros(len(wasp6paths_nodsub_odd))

    targetbounds = [385, 445]
    compbounds = [1390, 1460]
    roughnodcentroids = [500, 1500] # Rough indices between A and B nods
    apertureradius = 18#20#8
    bg_o = 5      # Background box outer limit
    bg_i = 1.5#2.5    # Background box inner limit

    ### Preparing channelshift() and bad pixel detections from badpixelsearch.py
    rowlimits = [5, 2030]
    collimits = [5, 2044]
    bestshiftspath = '/local/tmp/mosfire/2014sep18_analysis/bestxshifts.npy'
    bestxshifts = np.load(bestshiftspath)
    oversamplefactor = 1
    def channelshift(image):
        ydim, xdim = image.shape
        outputpaddingwidth = np.ceil(np.max(bestxshifts)/oversamplefactor)
        outputpadding = np.zeros((ydim, outputpaddingwidth))
        paddedimage = np.hstack([outputpadding, image, outputpadding])

        for row in range(1, ydim):
            paddedimage[row] = np.roll(paddedimage[row], int(bestxshifts[row]/oversamplefactor))
        return paddedimage

    rowlimits = [5, 2030]
    collimits = [5, 2044]
    shapeimg = pyfits.getdata('/local/tmp/mosfire/2014sep18/m140918_0005.fits')[rowlimits[0]:rowlimits[1],collimits[0]:collimits[1]]

    def channelshift_coords(x,y,dims=np.shape(shapeimg)):
        image = np.zeros(dims)
        image[y,x] = 1
        ydim, xdim = image.shape
        outputpaddingwidth = np.ceil(np.max(bestxshifts)/oversamplefactor)
        outputpadding = np.zeros((ydim, outputpaddingwidth))
        paddedimage = np.hstack([outputpadding, image, outputpadding])

        for row in range(1, ydim):
            paddedimage[row] = np.roll(paddedimage[row], int(bestxshifts[row]/oversamplefactor))
        XX, YY = np.meshgrid(range(ydim),range(xdim))
        #coords = (XX[paddedimage == 1][0], YY[paddedimage == 1][0])
        coords = (np.arange(ydim)[np.sum(paddedimage,axis=0) == 1][0], np.arange(xdim)[np.sum(paddedimage,axis=1) == 1][0])
        return coords

    badpxls_x = []
    badpxls_y = []
    badpxls_exp = []
    for basepath in ['badpxlseven/','badpxlsodd/']:
        badpxls_x.append(np.array(map(float, open(basepath+'variablepxls_x.csv').read().splitlines())))
        badpxls_y.append(np.array(map(float, open(basepath+'variablepxls_y.csv').read().splitlines())))
        badpxls_exp.append(np.array(map(float, open(basepath+'variablepxls_exp.csv').read().splitlines())))
    # for basepath in ['badpxlsodd/','badpxlseven/']:
    #     badpxls_x = np.array(map(float, open(basepath+'variablepxls_x.csv').read().splitlines()))
    #     badpxls_y = np.array(map(float, open(basepath+'variablepxls_y.csv').read().splitlines()))
    #     badpxls_exp = np.array(map(float, open(basepath+'variablepxls_exp.csv').read().splitlines()))

    badpxls_x = np.concatenate(badpxls_x)
    badpxls_y = np.concatenate(badpxls_y)
    badpxls_exp = np.concatenate(badpxls_exp)
    meaningfulinds = badpxls_exp != 1e10
    badpxls_x = badpxls_x[meaningfulinds]
    badpxls_y = badpxls_y[meaningfulinds]
    badpxls_exp = badpxls_exp[meaningfulinds]

    #################################################################################

    ## Galaxy image
    galaxyimage = np.load('/astro/users/bmmorris/git/research/keck/2014september/analysis/rightnod/galaxy/wholeframegalaxyimg.npy')
    galaxyimagesum = np.load('/astro/users/bmmorris/git/research/keck/2014september/analysis/rightnod/galaxy/wholeframegalaxysum.npy')

    # Expected transit time:
    t0_expected = 2456918.887816  # JD
    t0_roughfit = 2456918.8793039066
    t14duration_expected = 0.1086 # days

    Nbins = 8
    paddingbounds = [210, 2130]
    spectralbinbounds = np.linspace(paddingbounds[0], paddingbounds[1], Nbins+1, dtype=int)

    times = np.zeros(len(wasp6paths_nodsub_odd))
    apertureradii = [28]#np.arange(20,40)#[39]#np.arange(30,45)#np.arange(10, 70, 5)#np.arange(14, 25, 2)
    chisquared_allbins = np.zeros(len(apertureradii),dtype=float)
    # Fluxes/errors dimensions: 
    # N time series, N stars, N spectral bins, N apertures
    fluxes = np.zeros((len(wasp6paths_nodsub_odd), 2, Nbins, len(apertureradii)))
    errors = np.zeros_like(fluxes)
    centroids = np.zeros((len(wasp6paths_nodsub_odd), 2))
    airmass = np.zeros(len(wasp6paths_nodsub_odd))
    wavelengthbincenters = np.zeros(Nbins)
    exposuredurs = np.zeros(len(wasp6paths_nodsub_odd))

    # Additional bad pixel correction in core of PSF: 
    badpxlmap = np.load('/astro/users/bmmorris/git/research/keck/2014september/analysis/rightnod/badpxlincoremap.npy')
    def correctbp(image, badpxlmap, plots=False, copyimage=True, medianwindow=3):
        props = {'cmap':cm.Greys_r, 'origin':'lower', 'vmin':-1.7e6, 'vmax':1.7e6, 'interpolation':'nearest'}
        dims = np.shape(badpxlmap)
        XX, YY = np.meshgrid(np.arange(dims[1]), np.arange(dims[0]))
        if copyimage:
            correctedimage = image.copy()
        else:
            correctedimage = image

        if plots:
            fig, ax = plt.subplots(1, 2, figsize=(10,10), sharex=True, sharey=True)
            ax[0].imshow(image, **props)
            ax[0].plot(XX[badpxlmap], YY[badpxlmap], 'rx')

        for x, y in zip(XX[badpxlmap], YY[badpxlmap]):
            correctedimage[y, x] = np.median(np.concatenate([image[y, x-medianwindow:x], 
                                                             image[y, x+1:x+medianwindow]]))

        if plots:
            ax[1].imshow(correctedimage, **props)
            ax[1].plot(XX[badpxlmap], YY[badpxlmap], 'rx')
            plt.show(block=True)
        return correctedimage

    targetbounds = [385, 445]
    compbounds = [1390, 1460]
    roughnodcentroids = [500, 1500] # Rough indices between A and B nods
    #apertureradius = 18#20#8
    bg_o = 5      # Background box outer limit
    bg_i = 1.5#2.5    # Background box inner limit
    badpixelclip = 6.0 #sigma
    badpxlincore = 0
    #for i, imagepath in enumerate(wasp6paths_nodsub_odd[:1]):
    for i, imagepath, imagesumpath in zip(range(len(wasp6paths_nodsub_odd)), wasp6paths_nodsub_odd, wasp6paths_sum_odd):
        if i % 50 == 0: print i, 'of', len(wasp6paths_nodsub_odd)
        imagenameindex = int(imagepath.split('/')[-1].split('.')[0].split('_')[1].replace('n',''))
        image = pyfits.getdata(imagepath)#[:,paddingbounds[0]:paddingbounds[1]]
        header = pyfits.getheader(imagepath)
        imagesum = pyfits.getdata(imagesumpath)[:,paddingbounds[0]:paddingbounds[1]]
        times[i] = Time('2014-09-18 '+header['UTC'], scale='utc', format='iso').jd
        airmass[i] = header['AIRMASS']
        exposuredurs[i] = header['TRUITIME']

        ## Add in galaxy correction frame
        image += galaxyimage
        imagesum += galaxyimagesum[:,paddingbounds[0]:paddingbounds[1]]

        # crop image:
        image = correctbp(image, badpxlmap)[:,paddingbounds[0]:paddingbounds[1]]

        #image = image[:,paddingbounds[0]:paddingbounds[1]]

        for j in range(2):
            #target star is j=1
            leftcentroid, rightcentroid = photPack2.trackStarFine(image, [roughnodcentroids[j]-250,\
                                                               roughnodcentroids[j]+250], 0,\
                                                               plots=False, returnCentroidsOnly=True)
#print leftcentroid, rightcentroid
            if i % 2 == 0:
                centroids[i, j] = rightcentroid#leftcentroid
            else: 
                centroids[i, j] = leftcentroid

#             for k in range(Nbins):
#                 binimage = image[:, spectralbinbounds[k]:spectralbinbounds[k+1]]
#                 binimagesum = imagesum[:, spectralbinbounds[k]:spectralbinbounds[k+1]]
#                 wavelengthbincenters[k] = np.mean([wavelengthsoln[spectralbinbounds[k]], wavelengthsoln[spectralbinbounds[k+1]]])
#                 #print leftcentroid, rightcentroid
#                 midnod = np.mean([leftcentroid, rightcentroid])
#                 for l, apertureradius in enumerate(apertureradii):
#                     background_upper = binimage[centroids[i, j]+bg_i*apertureradius:centroids[i, j]+bg_o*apertureradius,:]
#                     background_lower = binimage[centroids[i, j]-bg_o*apertureradius:centroids[i, j]-bg_i*apertureradius,:]
#                     background = np.concatenate([background_upper, background_lower])#np.hstack([background_upper, background_lower])
#                     meanbackground = np.mean(background)
#                     rowprofile = np.sum(binimage[centroids[i, j]-10*apertureradius:centroids[i, j]+10*apertureradius,:],axis=1)
#                     withinaperture = binimage[centroids[i, j]-apertureradius:centroids[i, j]+apertureradius,:]
#                     withinaperture_sum = binimagesum[centroids[i, j]-apertureradius:centroids[i, j]+apertureradius,:]

#                     withinaperture_corrected = np.copy(withinaperture)
#                     medianwindow = 5
#                     corr_x = []
#                     corr_y = []
#                     lastlength = 0
#                     withinaperture = withinaperture_corrected

#                     fluxes[i, j, k, l] = (np.sum(withinaperture) - meanbackground*withinaperture.size)/exposuredurs[i]
#                     errors[i, j, k, l] = (np.sqrt(np.sum(withinaperture_sum) + meanbackground*withinaperture.size))/exposuredurs[i]
        #plt.plot(np.sum(image[leftcentroid-apertureradius:leftcentroid+apertureradius,:],axis=1))
    #plt.show()

    np.save('photoutputs_tmp/finecentroids.npy', centroids)
#     np.save('photoutputs_tmp/fluxes.npy', fluxes)
#     np.save('photoutputs_tmp/errors.npy', errors)
#     np.save('photoutputs_tmp/times.npy', times)
#     np.save('photoutputs_tmp/airmass.npy', airmass)
#     np.save('photoutputs_tmp/wavelengthbincenters.npy', wavelengthbincenters)
#     np.save('photoutputs_tmp/exposuredurs.npy', exposuredurs)
# else:
#     print 'Loading pre-calculated photometry'
#     fluxes = np.load('photoutputs_tmp/fluxes.npy')
#     errors = np.load('photoutputs_tmp/errors.npy')
#     times = np.load('photoutputs_tmp/times.npy')
#     airmass = np.load('photoutputs_tmp/airmass.npy')
#     wavelengthbincenters = np.load('photoutputs_tmp/wavelengthbincenters.npy')
#     exposuredurs = np.load('photoutputs_tmp/exposuredurs.npy')

Calculating photometry
0 of 264
50 of 264
100 of 264
150 of 264
200 of 264
250 of 264


In [38]:
reload(photPack2)

<module 'photPack2' from '../photPack2.pyc'>